## Almacenes de datos

Concepto
Los almacenes de datos de AzureML son abstracciones para fuentes de datos en la nube.

Funciones:
- Encapsulan información para la conexión a fuentes de datos.
- Permiten acceso directo a través del SDK de AzureML.
- Facilitan la carga y descarga de datos.

Tipos de Almacenes de Datos Soportados:
- Azure Blob Storage
- Azure File Storage
- Azure Data Lake Store
- Azure SQL Database
- Azure Databricks File System (DBFS)
- Entre los mas usados.. (Para la lista completa, consulte la documentación de AzureML)

Almacenes de Datos Predeterminados:
- Cada espacio de trabajo tiene dos almacenes predeterminados:
  - Contenedor Blob de Azure Storage
  - Contenedor de archivos de Azure Storage
- Se utilizan para el almacenamiento del sistema de AzureML.

Tercer Almacén de Datos:
Se agrega automáticamente al usar conjuntos de datos de ejemplo.

Uso en Proyectos:
- En la mayoría de los proyectos, necesitará usar sus propias fuentes de datos.Las razones pueden ser:
  1. Manejar mayores volúmenes de datos.
  2. Integrar la solución con datos de aplicaciones existentes.

## Crear y administrar Almacenes de datos

#### Registro:

- Interfaz gráfica de AzureML Studio.
- SDK de AzureML.

In [ ]:
# Registrar un contenedor Blob de Azure Storage llamado "blob_data"
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()

# Registramos el nuevo Datastore
blob_ds = Datastore.register_azure_blob_container(
    workspace=ws, 
    datastore_name='blob_data', 
    container_name='data', 
    account_name='storageaccountname', 
    container_name='data_container', 
    account_name='az_store...', 
    account_key='storageaccountkey'
    )

 #### Visualización y Gestión:

Realizable mediante:
- AzureML Studio.
- SDK de AzureML.

Ejemplo: Listar nombres de almacenes con un bucle for y obtener referencias individuales con el método get

In [ ]:
# Listamos los datastores registrados
for ds_name in ws.datastores:
    print(ds_name)

: 

#### Almacenamiento Predeterminado:

Incluido en todo espacio de trabajo que se crea.
- Recuperable con el método get_default_datastore.
- Inicialmente es "workspace blob store datastore".

In [ ]:
# Obtenemos el datastore recién registrado
blob_store = Datastore.get(ws, datastore_name='blob_data')

#### Consideraciones para la Planificación:

Azure Blob Storage:
- Almacenamiento premium: mejor rendimiento I/O para grandes conjuntos de datos (mayor costo y limitaciones de replicación).

Formato de Archivos:
- Parquet generalmente ofrece mejor rendimiento que CSV.

Acceso y Cambio de Predeterminado:
- Acceso por nombre del almacén de datos.
- Cambio de predeterminado con el método set_default_datastore.

In [ ]:
# configuramos el datastore por defecto
ws.set_default_datastore('blob_data')

### Ejercicio practico - trabajar con datasets

##### Creacion, carga y versiones
1. Creación de conjuntos de datos de archivos:
   
    Los datasets son objetos de datos empaquetados con versiones que se pueden usar en experimentos y pipelines. Son la forma recomendada para trabajar con datos en Azure Machine Learning.

    Tipos:

    - **Tabular**: Para datos con estructura consistente (ej. DataFrames de Pandas).
    - **Archivo**: Para datos no estructurados o procesamiento a nivel de archivo (ej. imágenes).

    El código en Python muestra cómo crear un dataset tabular a partir de dos rutas de archivo y registrarlo en el espacio de trabajo.1

In [ ]:
# Para crear un conjunto de datos tabular mediante el SDK, se usa el  método from_delimited_files de la  clase Dataset.Tabular, como se muestra a continuación:

from azureml.core import Dataset

blob_ds = ws.get_default_datastore()

csv_paths = [(blob_ds, 'data/files/current_data.csv'),          # Archivo current_data.csv en la  carpeta data/files/   
            (blob_ds, 'data/files/archive/*.csv')]              # Todos los archivos .csv en la  carpeta data/files/archive/

# Usando el método from_delimited_files
tab_ds = Dataset.Tabular.from_delimited_files(path=csv_paths)   # from_delimited_files de la  clase Dataset.Tabular

# 
tab_ds = tab_ds.register(workspace=ws, name='csv_table')

: 

In [ ]:
# Para crear un conjunto de datos de archivo mediante el SDK, use el  método from_files de la  clase Dataset.File

from azureml.core import Dataset

blob_ds = ws.get_default_datastore()

file_ds = Dataset.File.from_files(path=(blob_ds, 'data/files/images/*.jpg'))    # from_files de la  clase Dataset.File

file_ds = file_ds.register(workspace=ws, name='img_files')

: 

2. Recuperación de conjuntos de datos registrados:

    Se puede recuperar el dataset mediante cualquiera de las siguientes técnicas:

    - El  atributo `datasets` dictionary de un objeto Workspace.
    - Método `get_by_name` o `get_by_id` de la clase Dataset.

In [ ]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()                        # Obtenemos el objeto Workspace

ds1 = ws.datasets['csv_table']                      # Seleccionamos el conjunto de datos por su nombre 

ds2 = Dataset.get_by_name(ws, 'img_files')          # Seleccionamos el conjunto de datos por el nombre la su clase

3. Versionado de conjuntos de datos:

    Los conjuntos de datos se pueden versionar, lo que le permite realizar un seguimiento de las versiones históricas y reproducir esos experimentos con datos en el mismo estado.

    Puede crear una nueva versión de un conjunto de datos registrándolo con el mismo nombre que un conjunto de datos registrado anteriormente y especificando la  propiedad `create_new_version`

In [ ]:
# En este ejemplo, los archivos .png de la carpeta images se han agregado al conjunto de datos img_paths utilizado.

img_paths = [(blob_ds, 'data/files/images/*.jpg'),
             (blob_ds, 'data/files/images/*.png')]

file_ds = Dataset.File.from_files(path=img_paths)                                   # Usamos el método from_files para registrar los archivos jpg y png

file_ds = file_ds.register(workspace=ws, name='img_files', create_new_version=True) # creamos una nueva versión del conjunto de datos img_files

4. Recuperación de versiones específicas de conjuntos de datos:

    Podemos recuperar una versión específica de un conjunto especificando el parámetro version en el método `get_by_name` de la clase Dataset.
    

In [ ]:
img_ds = Dataset.get_by_name(workspace=ws, name='img_files', version=2)            # Obtenemos la versión 2 del conjunto de datos img_files

##### Manejo y uso

5. Trabajar con conjuntos de datos 

    Podemos leer datos directamente de un conjunto de datos tabular convirtiéndolos en un marco de datos de Pandas o Spark

In [ ]:

df = tab_ds.to_pandas_dataframe()
print(df.head()

6. Pasar un conjunto de datos tabular a un script
   
Cuando necesitemos acceder a un conjunto de datos mediante un script, debemos pasarle el conjunto de datos al script. 

Hay dos maneras de hacerlo:

- **Usar un argumento de script para un conjunto de datos tabulares**

    Podemos pasar el conjunto de datos como argumento de script. 
    
    Cuando se adopta este enfoque, el argumento recibido por el script es el identificador único del conjunto de datos en el área de trabajo. En el script, puede obtener el área de trabajo del contexto de ejecución y usarlo para recuperar el conjunto de datos por su identificador.

In [ ]:
# ScriptRunConfig

env = Environment('my_env')                                                         # Creamos un objeto Environment  
packages = CondaDependencies.create(conda_packages=['pip'],                         # Creamos un objeto CondaDependencies con las dependencias necesarias
                                    pip_packages=['azureml-defaults',
                                                  'azureml-dataprep[pandas]'])
env.python.conda_dependencies = packages                                            # Asignamos las dependencias al entorno                             

script_config = ScriptRunConfig(source_directory='my_dir',                          # Creamos un objeto ScriptRunConfig   
                                script='script.py',
                                arguments=['--ds', tab_ds],)

In [ ]:
# Script

from azureml.core import Run, Dataset

parser.add_argument('--ds', type=str, dest='dataset_id')                            # Agregamos un argumento para el conjunto de datos
args = parser.parse_args()

run = Run.get_context()
ws = run.experiment.workspace
dataset = Dataset.get_by_id(ws, id=args.dataset_id)                                 # Obtenemos el conjunto de datos por su id
data = dataset.to_pandas_dataframe()

- **Usar una entrada con nombre para un conjunto de datos tabular**

    Como alternativa, podemos pasar un conjunto de datos tabular como una entrada con nombre. En este enfoque, se utiliza el  método `as_named_input` del conjunto de datos para especificar un nombre. 

    A continuación, en el script, puede recuperar el conjunto de datos por nombre de la colección `input_datasets` del contexto de ejecución sin necesidad de recuperarlo del área de trabajo. Tenga en cuenta que si usa este enfoque, aún debe incluir un argumento de script para el conjunto de datos, aunque en realidad no lo use para recuperar el conjunto de datos.

In [ ]:
#ScriptRunConfig

env = Environment('my_env')
packages = CondaDependencies.create(conda_packages=['pip'],
                                    pip_packages=['azureml-defaults',
                                                  'azureml-dataprep[pandas]'])
env.python.conda_dependencies = packages

script_config = ScriptRunConfig(source_directory='my_dir',
                                script='script.py',
                                arguments=['--ds', tab_ds.as_named_input('my_dataset')],    # Pasamos el conjunto de datos como argumento
                                environment=env)

In [ ]:
# Script

from azureml.core import Run

parser.add_argument('--ds', type=str, dest='ds_id')
args = parser.parse_args()

run = Run.get_context()
dataset = run.input_datasets['my_dataset']                                          # Obtenemos el conjunto de datos por su nombre
data = dataset.to_pandas_dataframe()

7. Trabajar con conjuntos de datos de archivos

    Para trabajar con un conjunto de datos de archivos, podemos utilizar el método `to_path()` para devolver una lista de las rutas de archivo encapsuladas por el conjunto de datos

In [ ]:
for file_path in file_ds.to_path():
    print(file_path)

8. Pasar un conjunto de datos de archivo a un script 

    Al igual que con los conjuntos de datos tabulares, hay dos formas de pasar un conjunto de datos de archivo a un script. Sin embargo, hay algunas diferencias clave en la forma en que se pasa el conjunto de datos.

    - **Usar un argumento de script para un conjunto de datos de archivos**

        Puede pasar un conjunto de datos de archivos como argumento de script. A diferencia de lo que ocurre con un conjunto de datos tabular, debe especificar un modo para el argumento del conjunto de datos de archivo, que puede ser `as_download` o `as_mount`. Esto proporciona un punto de acceso que el script puede utilizar para leer los archivos del conjunto de datos. 

        En la mayoría de los casos, se ha de `as_download`, que copia los archivos en una ubicación temporal en el proceso donde se ejecuta el script. Sin embargo, si está trabajando con una gran cantidad de datos para los que es posible que no haya suficiente espacio de almacenamiento en el proceso, usaremos `as_mount` para transmitir los archivos directamente desde su origen.

In [ ]:
#ScriptRunConfig

env = Environment('my_env')
packages = CondaDependencies.create(conda_packages=['pip'],
                                    pip_packages=['azureml-defaults',
                                                  'azureml-dataprep[pandas]'])
env.python.conda_dependencies = packages

script_config = ScriptRunConfig(source_directory='my_dir',
                                script='script.py',
                                arguments=['--ds', file_ds.as_download()],                          # Pasamos el conjunto de datos como argumento usando el método as_download
                                environment=env)

In [ ]:
#Script

from azureml.core import Run
import glob

parser.add_argument('--ds', type=str, dest='ds_ref')
args = parser.parse_args()
run = Run.get_context()

imgs = glob.glob(args.ds_ref + "/*.jpg")                                                            # Obtenemos los archivos del conjunto de datos

- **Usar una entrada con nombre para un conjunto de datos de archivos**

    También puede pasar un conjunto de datos de archivos como una entrada con nombre. En este enfoque, se utiliza el  método as_named_input del conjunto de datos para especificar un nombre antes de especificar el modo de acceso. A continuación, en el script, puede recuperar el conjunto de datos por nombre de la colección input_datasets del contexto de ejecución  y leer los archivos desde allí. 

    Al igual que con los conjuntos de datos tabulares, si usa una entrada con nombre, debe incluir un argumento de script para el conjunto de datos, aunque en realidad no lo use para recuperar el conjunto de datos.

In [ ]:
# ScriptRunConfig

env = Environment('my_env')
packages = CondaDependencies.create(conda_packages=['pip'],
                                    pip_packages=['azureml-defaults',
                                                  'azureml-dataprep[pandas]'])
env.python.conda_dependencies = packages

script_config = ScriptRunConfig(source_directory='my_dir',
                                script='script.py',
                                arguments=['--ds', file_ds.as_download()],                       # Pasamos el conjunto de datos como argumento usando el método as_download
                                environment=env)

In [ ]:
# Script

from azureml.core import Run
import glob

parser.add_argument('--ds', type=str, dest='ds_ref')
args = parser.parse_args()
run = Run.get_context()

imgs = glob.glob(args.ds_ref + "/*.jpg")                                                           # Obtenemos los archivos del conjunto de datos

11. Usar una entrada con nombre para un conjunto de datos de archivos
    
    En este enfoque, se utiliza el  método `as_named_input` del conjunto de datos para especificar un nombre antes de especificar el modo de acceso. A continuación, en el script, puede recuperar el conjunto de datos por nombre de la colección `input_datasets` del contexto de ejecución y leer los archivos desde allí. 

    Al igual que con los conjuntos de datos tabulares, si usa una entrada con nombre, debe incluir un argumento de script para el conjunto de datos, aunque en realidad no lo use para recuperar el conjunto de datos.

In [ ]:
# ScriptRunConfig

env = Environment('my_env')
packages = CondaDependencies.create(conda_packages=['pip'],
                                    pip_packages=['azureml-defaults',
                                                  'azureml-dataprep[pandas]'])
env.python.conda_dependencies = packages

script_config = ScriptRunConfig(source_directory='my_dir',
                                script='script.py',
                                arguments=['--ds', file_ds.as_named_input('my_ds').as_download()],    # Pasamos el conjunto de datos como argumento usando el método as_named_input
                                environment=env)

In [ ]:
# Script

from azureml.core import Run
import glob

parser.add_argument('--ds', type=str, dest='ds_ref')
args = parser.parse_args()
run = Run.get_context()

dataset = run.input_datasets['my_ds']                                                        # Obtenemos el conjunto de datos por su nombre 
imgs= glob.glob(dataset + "/*.jpg")

: 